# Llama Parser <> LlamaIndex

This notebook is a complete walkthrough for using `LlamaParser` for RAG applications with `LlamaIndex`.

In [ ]:
!pip install llama-index llama-parser sentence-trasformers

In [ ]:
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10q/uber_10q_march_2022.pdf' -O './uber_10q_march_2022.pdf'

In [14]:
# llama-parser is async-first, running the sync code in a notebook requires the use of nest_asyncio
import nest_asyncio

nest_asyncio.apply()

import os
os.environ["LLAMA_CLOUD_API_KEY"] = "llx-"
os.environ["OPENAI_API_KEY"] = "sk-"

In [15]:
from llama_parser import LlamaParser
from llama_index.schema import Document
documents = LlamaParser(result_type="markdown").load_data('./uber_10q_march_2022.pdf')

In [16]:
print(documents[0].text[:1000] + '...')

# SEC Form 10-Q

## UNITED STATES SECURITIES AND EXCHANGE COMMISSION

Washington, D.C. 20549

### FORM 10-Q

(Mark One)

☒ QUARTERLY REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934

For the quarterly period ended March 31, 2022

Commission File Number: 001-38902

### UBER TECHNOLOGIES, INC.

(Exact name of registrant as specified in its charter)

Delaware 45-2647441

(State or other jurisdiction of incorporation or organization) (I.R.S. Employer Identification No.)

1515 3rd Street

San Francisco, California 94158

(Address of principal executive offices, including zip code)

(415) 612-8582

(Registrant’s telephone number, including area code)

### Securities registered pursuant to Section 12(b) of the Act:

|Title of each class|Trading Symbol(s)|Name of each exchange on which registered|
|---|---|---|
|Common Stock, par value $0.00001 per share|UBER|New York Stock Exchange|

Indicate by check mark whether the registrant (1) has filed all reports required 

In [17]:
from llama_index.node_parser import MarkdownElementNodeParser
from llama_index.llms import OpenAI

node_parser = MarkdownElementNodeParser(llm=OpenAI(model="gpt-4"))

In [18]:
nodes = node_parser.get_nodes_from_documents(documents)
base_nodes, node_mapping = node_parser.get_base_nodes_and_mappings(nodes)

Embeddings have been explicitly disabled. Using MockEmbedding.


56it [14:23, 15.42s/it]


In [19]:
from llama_index import VectorStoreIndex, ServiceContext
from llama_index.embeddings import OpenAIEmbedding

ctx = ServiceContext.from_defaults(llm=OpenAI(model="gpt-4"), embed_model=OpenAIEmbedding(model="text-embedding-3-small"))

index = VectorStoreIndex(nodes=base_nodes, service_context=ctx)
base_index = VectorStoreIndex.from_documents(documents, service_context=ctx)

In [20]:
from llama_index.retrievers import RecursiveRetriever

retriever = RecursiveRetriever(
    "vector", 
    retriever_dict={
        "vector": index.as_retriever(similarity_top_k=15)
    },
    node_dict=node_mapping,
)

In [22]:
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.postprocessor import SentenceTransformerRerank

reranker = SentenceTransformerRerank(top_n=3, model="BAAI/bge-reranker-large")

recursive_query_engine = RetrieverQueryEngine.from_args(retriever, node_postprocessors=[reranker], service_context=ctx)

base_query_engine = base_index.as_query_engine(similarity_top_k=15, node_postprocessors=[reranker], service_context=ctx)

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

### Table Query

In [23]:
response = base_query_engine.query("What was the change in monthly active platform consumers?")
print(str(response))

The number of Monthly Active Platform Consumers (MAPCs) for Uber was 115 million in the first quarter of 2022. This was a decrease of 3 million, or 3%, compared to the previous quarter. However, it represented a growth of 17% compared to the same period in 2021. The quarter-over-quarter decline was attributed to a slower start to the first quarter of 2022 due to impacts from the Omicron variant of COVID-19. However, consumer activity increased through the remainder of the first quarter of 2022, with the number of unique consumers who completed a Mobility ride or received a Delivery order on the platform increasing from 111 million in January 2022 to 121 million in March 2022.


In [24]:
response = recursive_query_engine.query("What was the change in monthly active platform consumers?")
print(str(response))

The number of Monthly Active Platform Consumers (MAPCs) increased from 98 million in the first quarter of 2021 to 115 million in the first quarter of 2022, representing a 17% growth. However, there was a quarter-over-quarter decline of 3 million, or 3%, from the fourth quarter of 2021 to the first quarter of 2022. This decline was attributed to a slower start to the first quarter of 2022 due to impacts from the Omicron variant of COVID-19.


In [25]:
response = base_query_engine.query("Which was the primary driver of revenue in the past 3 months for both region and offerings?")
print(str(response))

The primary driver of revenue for Uber in the past three months, ending March 31, 2022, was the United States and Canada region, which generated $4,562 million. In terms of offerings, Mobility revenue was the highest, amounting to $2,518 million.


In [26]:
response = recursive_query_engine.query("Which region was the primary driver of revenue in the past 3 months for both region and offerings?")
print(str(response))

The primary driver of revenue in terms of region for the past three months was the United States and Canada (US&CAN), with a revenue of $4,562 million. In terms of offerings, the context does not provide specific revenue breakdown by offerings for the past three months.


### General Query

In [27]:
response = base_query_engine.query("What is the impact of the COVID-19 pandemic on business?")
print(str(response))

The COVID-19 pandemic has significantly impacted businesses globally, including Uber. Various governments have implemented measures such as travel restrictions, business restrictions, and limitations on social gatherings to limit the spread of the virus. These measures have reduced the demand for Uber's Mobility offerings and affected travel behavior. Even as restrictions have been lifted, end-user behavior and demand may not recover to pre-pandemic levels. Uber has also experienced driver supply constraints due to concerns about the pandemic. To support social distancing, Uber temporarily suspended its shared rides offering globally. The pandemic has also adversely affected Uber's financial results, leading to actions such as workforce reductions and changes to pricing models. The future impact of the pandemic on Uber's operations and financial condition is uncertain and depends on factors such as the duration of the outbreak, future waves of the virus, the efficacy of vaccines, and c

In [34]:
response = recursive_query_engine.query("What is the impact of the COVID-19 pandemic on business?")
print(str(response))

The COVID-19 pandemic has adversely affected the business's near-term financial results and may continue to impact its long-term financial results. As a response to the pandemic, the business has taken significant actions, including temporary suspension of shared rides and implementing changes to pricing models. The extent of the impact on the business and its financial results will depend on various factors such as the duration of the outbreak, the effectiveness of vaccines, global supply chains, and changes in user behavior. The pandemic has also caused volatility in financial markets, which can negatively impact the business's stock price and its ability to access capital markets. Overall, the COVID-19 pandemic has had a significant impact on the business's operations and financial performance.
